<a href="https://colab.research.google.com/github/dhruthick/cse256project/blob/main/recommendation/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Collaborative Filtering for Recommendation

## Imports

In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from collections import defaultdict

## Read Data

In [2]:
data_path='/content/drive/MyDrive/cse256/project/data/'
interactions_train=pd.read_csv(data_path+'interactions_train.csv')

In [7]:
val=pd.read_csv(data_path+'interactions_val.csv')

Some essential data structures

In [3]:
playlistsPerTrack = defaultdict(set) 
tracksPerPlaylist = defaultdict(set) 
trackNames = {}
interactionMatrix = {}

for playlist,track,track_name,artist in tqdm(interactions_train[['pid','track_uri','track_name','artist_name']].values.tolist()):
    playlistsPerTrack[track].add(playlist)
    tracksPerPlaylist[playlist].add(track)
    interactionMatrix[(playlist,track)] = 1
    trackNames[track] = (track_name, artist)

100%|██████████| 350200/350200 [00:01<00:00, 287772.06it/s]


## Utility Functions

Function to calculate jaccard similarity between two sets

In [4]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

Function to calculate similarity scores for tracks to be recommended given a single track

In [5]:
def mostSimilarFast(track, return_all=False, N=10):
    similarities = []
    playlists = playlistsPerTrack[track]
    candidateTracks = set()
    for p in playlists:
        candidateTracks = candidateTracks.union(tracksPerPlaylist[p])
    for t in candidateTracks:
        if t == track: continue
        sim = Jaccard(playlists, playlistsPerTrack[t])
        similarities.append((sim,t,trackNames[t]))
    similarities.sort(reverse=True)
    if return_all:
      return similarities
    return similarities[:N]

Function to fetch recommedend tracks and their average similarity scores given a list of tracks

In [6]:
def recommend_tracks(tracks,N):
  scores=defaultdict(float)
  num_tracks=len(tracks)
  for track in tracks:
    similarities=mostSimilarFast(track, return_all=True)
    for sim,t,name in similarities:
      if t not in tracks:
        scores[t]+=sim
  for k in scores.keys():
    scores[k]/=num_tracks
  scores=sorted(scores.items(), key=lambda item: item[1], reverse=True)
  return scores[:N]

## Evaluation

Function to calculate R-Precision (fraction of relevant songs that were recommended), NDCG score, and Recommendation Clicks (number of clicks until the first relevant track is encountered, where each click fetches 10 new track recommendations)

In [17]:
def evaluate_playlist_rec(pid,N=500):
  tracksInPlaylist=tracksPerPlaylist[pid]
  relevantTracks=set(val[val['pid']==pid].track_uri.values)
  scores=recommend_tracks(tracksInPlaylist,N=N)
  recommendedTracks=set([t[0] for t in scores])
  rprc=len(recommendedTracks.intersection(relevantTracks))/len(relevantTracks)
  dcg=0
  for i in range(len(scores)):
    if scores[i][0] in relevantTracks:
      dcg+=math.log(2)/math.log(i+2)
  ndcg=dcg/len(relevantTracks)
  rec_click=51
  for i in range(0,50):
    recommendedTracks=set([t[0] for t in scores[i*10:(i*10+10)]])
    if len(recommendedTracks.intersection(relevantTracks))>0:
      rec_click=i+1
      break
  rec_click/=10
  return rprc,ndcg,rec_click

In [20]:
playlists=np.unique(val.pid.values)
rprcs,ndcgs,rec_clicks=[],[],[]
for pid in tqdm(playlists):
  rprc,ndcg,rec_click=evaluate_playlist_rec(pid,N=500)
  rprcs.append(rprc)
  ndcgs.append(ndcg)
  rec_clicks.append(rec_click)

print(f'\nAverage R-Precision: {np.average(rprcs)}')
print(f'Average NDCG: {np.average(ndcgs)}')
print(f'Average Recommendation Clicks: {np.average(rec_clicks)}')

100%|██████████| 7476/7476 [53:52<00:00,  2.31it/s]


Average R-Precision: 0.4100231934245089
Average NDCG: 0.0875240137148912
Average Recommendation Clicks: 1.540181915462814
